In [ ]:
# prima della fase di ETL, ho fatto alcune modifiche ai file con OpenRefine:
# TAB1.hotels = valori numerici di price e rate_num, separati alcuni campi (lat e long per Kepler)
# TAB2.facilities = no NULL (perchè lo split di una lista vuota non restituisce valori)

In [ ]:
# con pandas (ETL) vado a "pulire" e mettere in qualità la lista di hotel raccolti in first page (tab1),
# poi faccio la stessa cosa con le facilities raccolte in dettaglio (tab2)
# carico le due tabelle "db_tablename" in SQLite per lo storage
# farò le join (chiave = hotel_id) e le queries SQLite e proverò a fare una merge su python w/pandas

# ETL - HOTELS

In [28]:
from google.colab import files
file = files.upload()

Saving hotels_facilities_detail.csv to hotels_facilities_detail.csv


In [3]:
import pandas as pd
hotels = pd.read_csv("hotels_mod.csv", index_col="hotel_id")
hotels.head(20)

,Column,url,name,rating,rate_num,long,lat,score,reviews,breakfast,canc_policy,payment_policy,price,currency,price_num
hotel_id,,,,,,,,,,,,,,,
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,0,https://www.booking.com/hotel/it/cala-di-seta....,Hotel Cala Di Seta,3 out of 5,3.0,8.371029,39.110008,8.4,429.0,Colazione inclusa,Cancellazione GRATUITA,• Senza pagamento anticipato,€ 808,€,808.0
https://www.booking.com/hotel/it/lemon-tree-quartu-sant-39-elena.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=622749606_310708075_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=2&highlighted_blocks=622749606_310708075_2_1_0&hpos=2&no_rooms=1&sr_order=popularity&sr_pri_blocks=622749606_310708075_2_1_0__55394&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=&tpi_r=2#hotelTmpl,1,https://www.booking.com/hotel/it/lemon-tree-qu...,Lemontree QSE,4 out of 5,4.0,9.183331,39.241235,9.4,29.0,Colazione inclusa,Cancellazione GRATUITA,NaN,€ 554,€,554.0
https://www.booking.com/hotel/it/petri-marini.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=25090906_295459154_0_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=3&highlighted_blocks=25090906_295459154_0_1_0&hpos=3&no_rooms=1&sr_order=popularity&sr_pri_blocks=25090906_295459154_0_1_0__85399&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=&tpi_r=2#hotelTmpl,2,https://www.booking.com/hotel/it/petri-marini....,Hotel Petri Marini,3 out of 5,3.0,9.064904,41.124956,8.9,188.0,Colazione inclusa,Cancellazione GRATUITA,• Senza pagamento anticipato,€ 854,€,854.0
https://www.booking.com/hotel/it/residenze-soledade.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=182967303_204658941_4_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=4&highlighted_blocks=182967303_204658941_4_1_0&hpos=4&no_rooms=1&sr_order=popularity&sr_pri_blocks=182967303_204658941_4_1_0__146300&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=&tpi_r=2#hotelTmpl,3,https://www.booking.com/hotel/it/residenze-sol...,Residenze Soledade,3 out of 5,3.0,9.495513,40.898161,8.5,271.0,Colazione inclusa,Cancellazione GRATUITA,NaN,€ 1463,€,1463.0
https://www.booking.com/hotel/it/arbatax-park-resort-cottage-del-mare.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=24221101_226631340_0_1_0&checkin=2021-07-25&checkout=2021-08-0

In [4]:
len(hotels)

1039

In [5]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1039 entries, https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl to https://www.booking.com/hotel/it/camping-capo-d-39-orso.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid

In [6]:
# DA FARE:
# > Eliminare DUPLICATI per "name"
# > Gestione dei NULL, in particolare per:
  # - "price": da eliminare perchè sono pochi ed è valore chiave per le analisi;
  # - "rating": non è un problema, posso filtrare anche i senza rating --> sostituisco il NULL con "No Rating"
  # - "score": non li elimino, imputo il valore della media degli score, poi arrotondo
# > Inserire COLONNA CON CLASSE DI PREZZO

In [7]:
# da OpenRefine vedo dei NOMI duplicati --> verifico e li elimino con Python
hotels['name'].duplicated().sum()

44

In [8]:
hotels = hotels.drop_duplicates(subset='name')
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 995 entries, https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl to https://www.booking.com/hotel/it/camping-capo-d-39-orso.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=

In [9]:
# dalle info sopra e anche da OR vedo che ci sono dei valori NULL in:
# - rating = non rappresenta un problema per me, perchè poi posso comunque farci dei filtri
# - price = ALERT! devo gestire questi valori mancanti perchè nelle query prezzo medio impattano!!!
hotels['price'].isnull().sum()

16

In [10]:
# sono pochi, li elimino perchè non posso imputare nessun valore
hotels = hotels[hotels['price'].notnull()]
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 979 entries, https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl to https://www.booking.com/hotel/it/camping-capo-d-39-orso.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=

In [ ]:
# vedo da info e OR che ci sono alcuni hotel con "score"=NULL
# non li elimino, ma posso gestirli imputando la media per rating
# SE NON RIESCO, imputo la media della variabile "score", senza considerare i singoli rating... 
# vedo che non cambia molto: la media tot è 8.6, in linea con gli altri valori osservati

In [11]:
hotels['rating'].unique()

array(['3 out of 5', '4 out of 5', nan, '2 out of 5', '1 out of 5',
       '5 out of 5'], dtype=object)

In [12]:
mean = hotels.groupby('rating')['score'].mean()
print(mean)

rating
1 out of 5    7.200000
2 out of 5    8.280000
3 out of 5    8.621795
4 out of 5    8.413661
5 out of 5    8.825000
Name: score, dtype: float64


In [13]:
hotels['rating'].isnull().sum()

217

In [14]:
# non mi attribuisce la media per il valore NULL del rating
# sostituisco al rating=NULL il testo "NO RATING", così lo riconosce come valore
import numpy as np
hotels['rating'] = hotels['rating'].fillna('No Rating')
hotels['rating'].unique()

array(['3 out of 5', '4 out of 5', 'No Rating', '2 out of 5',
       '1 out of 5', '5 out of 5'], dtype=object)

In [15]:
mean = hotels.groupby('rating')['score'].mean().round(1)
print(dict(mean))
target = dict(mean)
print(target)

{'1 out of 5': 7.2, '2 out of 5': 8.3, '3 out of 5': 8.6, '4 out of 5': 8.4, '5 out of 5': 8.8, 'No Rating': 8.7}
{'1 out of 5': 7.2, '2 out of 5': 8.3, '3 out of 5': 8.6, '4 out of 5': 8.4, '5 out of 5': 8.8, 'No Rating': 8.7}


In [16]:
hotels['score'].isnull().sum()

48

In [17]:
hotels['score'].mean()

8.587862513426415

In [18]:
hotels['score'] = hotels['score'].fillna(hotels['score'].mean())
hotels['score'].value_counts()

9.000000     59
8.700000     51
8.900000     48
8.587863     48
8.800000     46
9.200000     45
8.500000     45
9.300000     44
9.100000     43
9.400000     42
8.400000     41
8.300000     40
8.000000     39
8.600000     38
9.500000     38
8.100000     37
8.200000     31
7.900000     31
9.600000     26
7.800000     24
7.700000     21
9.700000     18
7.500000     16
7.000000     14
7.600000     14
7.400000     13
10.000000    12
9.800000     10
7.200000      8
6.900000      8
7.300000      7
7.100000      4
6.800000      4
6.000000      2
6.600000      2
5.900000      2
6.700000      2
9.900000      2
5.000000      1
4.500000      1
5.700000      1
5.200000      1
Name: score, dtype: int64

In [19]:
hotels['score'] = hotels['score'].round(1)
hotels['score'].value_counts()

8.6     86
9.0     59
8.7     51
8.9     48
8.8     46
9.2     45
8.5     45
9.3     44
9.1     43
9.4     42
8.4     41
8.3     40
8.0     39
9.5     38
8.1     37
7.9     31
8.2     31
9.6     26
7.8     24
7.7     21
9.7     18
7.5     16
7.0     14
7.6     14
7.4     13
10.0    12
9.8     10
7.2      8
6.9      8
7.3      7
7.1      4
6.8      4
6.0      2
6.6      2
5.9      2
6.7      2
9.9      2
5.0      1
4.5      1
5.7      1
5.2      1
Name: score, dtype: int64

In [ ]:
# da ultimo, creo la colonna della "price_class" sulla base della classificazione data

In [20]:
import numpy as np
hotels['price_class'] = np.where(hotels['price_num'] < 500, '< 500',
                                 np.where(hotels['price_num'] <= 1000, '500 - 1.000',
                                          np.where(hotels['price_num'] <= 1500, '1.000 - 1.500',
                                                   np.where(hotels['price_num'] <= 2000, '1.500 - 2.000', '> 2.000'))))
hotels['price_class'].value_counts()

500 - 1.000      476
< 500            217
1.000 - 1.500    151
> 2.000           69
1.500 - 2.000     66
Name: price_class, dtype: int64

In [22]:
hotels.head(10)

,Column,url,name,rating,rate_num,long,lat,score,reviews,breakfast,canc_policy,payment_policy,price,currency,price_num,price_class
hotel_id,,,,,,,,,,,,,,,,
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,0,https://www.booking.com/hotel/it/cala-di-seta....,Hotel Cala Di Seta,3 out of 5,3.0,8.371029,39.110008,8.4,429.0,Colazione inclusa,Cancellazione GRATUITA,• Senza pagamento anticipato,€ 808,€,808.0,500 - 1.000
https://www.booking.com/hotel/it/lemon-tree-quartu-sant-39-elena.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=622749606_310708075_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=2&highlighted_blocks=622749606_310708075_2_1_0&hpos=2&no_rooms=1&sr_order=popularity&sr_pri_blocks=622749606_310708075_2_1_0__55394&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=&tpi_r=2#hotelTmpl,1,https://www.booking.com/hotel/it/lemon-tree-qu...,Lemontree QSE,4 out of 5,4.0,9.183331,39.241235,9.4,29.0,Colazione inclusa,Cancellazione GRATUITA,NaN,€ 554,€,554.0,500 - 1.000
https://www.booking.com/hotel/it/petri-marini.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=25090906_295459154_0_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=3&highlighted_blocks=25090906_295459154_0_1_0&hpos=3&no_rooms=1&sr_order=popularity&sr_pri_blocks=25090906_295459154_0_1_0__85399&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=&tpi_r=2#hotelTmpl,2,https://www.booking.com/hotel/it/petri-marini....,Hotel Petri Marini,3 out of 5,3.0,9.064904,41.124956,8.9,188.0,Colazione inclusa,Cancellazione GRATUITA,• Senza pagamento anticipato,€ 854,€,854.0,500 - 1.000
https://www.booking.com/hotel/it/residenze-soledade.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=182967303_204658941_4_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=4&highlighted_blocks=182967303_204658941_4_1_0&hpos=4&no_rooms=1&sr_order=popularity&sr_pri_blocks=182967303_204658941_4_1_0__146300&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=&tpi_r=2#hotelTmpl,3,https://www.booking.com/hotel/it/residenze-sol...,Residenze Soledade,3 out of 5,3.0,9.495513,40.898161,8.5,271.0,Colazione inclusa,Cancellazione GRATUITA,NaN,€ 1463,€,1463.0,1.000 - 1.500
https://www.booking.com/hotel/it/arbatax-park-resort-cottage-del-mare.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks

In [34]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 979 entries, https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl to https://www.booking.com/hotel/it/camping-capo-d-39-orso.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=

In [35]:
hotels.to_csv("db_hotels.csv")

In [ ]:
# il db è pronto per l'esportazione in csv e il caricamento in MySQL

# ETL - FACILITIES

In [ ]:
# Ora metto in qualità il file delle facilities ed esporto il csv def

In [29]:
import pandas as pd
facilities = pd.read_csv("hotels_facilities_detail.csv", index_col='hotel_id')
facilities.head()

,Column,url,item
hotel_id,,,
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,0,https://www.booking.com/hotel/it/cala-di-seta....,Camere non fumatori
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,1,https://www.booking.com/hotel/it/cala-di-seta....,Spa & centro benessere
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,2,https://www.booking.com/hotel/it/cala-di-seta....,Parcheggio gratuito
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,3,https://www.booking.com/hotel/it/cala-di-seta....,Animali ammessi
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,4,https://www.booking.com/hotel/it/cala-di-seta....,Connessione WiFi gratuita


In [ ]:
facilities['item'].value_counts()

Camere non fumatori                                    630
Connessione WiFi gratuita                              557
Parcheggio gratuito                                    522
Bar                                                    491
Animali ammessi                                        307
1 piscina                                              228
Disponibilità di camere familiari                      216
Ristorante                                             209
Colazione buona                                        140
Camere/strutture per ospiti disabili                   137
Navetta aeroportuale                                   132
bollitore tè / macchina caffè in tutte le camere       129
Fronte spiaggia                                        127
Servizio in camera                                     125
Colazione ottima                                       125
Colazione                                              110
Colazione eccezionale                                  1

In [ ]:
list(facilities['item'].unique())

['Camere non fumatori',
 'Spa & centro benessere',
 'Parcheggio gratuito',
 'Animali ammessi',
 'Connessione WiFi gratuita',
 'Bar',
 'Colazione eccezionale',
 'Colazione eccellente',
 'Navetta aeroportuale',
 'WiFi disponibile ovunque',
 '2 piscine',
 'Spa & centro benessere (al momento non disponibile)',
 'Ristorante',
 'Fronte spiaggia',
 'Camere/strutture per ospiti disabili',
 'Colazione buona',
 'bollitore tè / macchina caffè in tutte le camere',
 '1 piscina',
 'Palestra',
 'Servizio in camera',
 'Disponibilità di camere familiari',
 'Reception 24 ore su 24',
 'Colazione',
 'Riscaldamento',
 'Spiaggia privata',
 'Colazione ottima',
 'Colazione favolosa',
 '3 piscine',
 'Parcheggio',
 'palestra eccellente',
 'Parcheggio privato',
 'Barbecue',
 'palestra buona',
 'palestra favolosa',
 'palestra eccezionale',
 'Terrazza',
 'Giardino',
 'Bar (al momento non disponibile)',
 'palestra ottima',
 'WiFi',
 'Ristorante (al momento non disponibile)']

In [30]:
facilities['item'].isnull().sum()

0

In [36]:
facilities.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4856 entries, https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl to https://www.booking.com/hotel/it/camping-capo-d-39-orso.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid

In [38]:
facilities.drop('url', axis=1)

,Column,item
hotel_id,,
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,0,Camere non fumatori
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,1,Spa & centro benessere
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,2,Parcheggio gratuito
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,3,Animali ammessi
https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl,4,Connessione WiFi gratuita
...,...,...
https://www.booking.com/hotel/it/camping-capo-d-39-orso.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=107780503_226709626_0_2_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1001&highlighted_blocks=107780503_226709626_0_2_0&hpos=26&no_rooms=

In [ ]:
facilities = facilities.drop('Column', axis=1)

In [41]:
facilities = facilities.drop('url', axis=1)

In [44]:
facilities.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4856 entries, https://www.booking.com/hotel/it/cala-di-seta.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid=82e53658004e6000ce5045a102ed928f&all_sr_blocks=18644307_309892242_2_1_0&checkin=2021-07-25&checkout=2021-08-01&dest_id=908&dest_type=region&group_adults=2&group_children=0&hapos=1&highlighted_blocks=18644307_309892242_2_1_0&hpos=1&no_rooms=1&sr_order=popularity&sr_pri_blocks=18644307_309892242_2_1_0__80800&srepoch=1625766956&srpvid=39147e150e670145&ucfs=1&from=searchresults;highlight_room=#hotelTmpl to https://www.booking.com/hotel/it/camping-capo-d-39-orso.it.html?aid=376372&label=it-5Srxg0e1twJI_ryrey2UnQS267778030990%3Apl%3Ata%3Ap1%3Ap22.563.000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1008463%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YVcLb15uXY9drDRMbmnr9EE&sid

In [45]:
# rispetto alla versione con facilities_list, splittando ogni singolo elemento NON prendo i valori nulli
# il dato è in qualità


In [46]:
facilities.to_csv('db_facilities.csv')